In [1]:
from sqlalchemy import create_engine
import pandas as pd
import os
# Replace the following values with your actual database
username = 'postgres'
password = 'ffzhzicfpkpsbljjdsdwlawqdklenfsfzchtknhpwkpndyzoxq'
host = 'srv17.mikr.us'
port = '30165'  # Changed to match the port Docker is listening on
database = 'postgres'

engine = create_engine(f'postgresql://{username}:{password}@{host}:{port}/{database}')

os.environ['DB_CONNECTION_STRING'] = f'postgresql://{username}:{password}@{host}:{port}/{database}'
os.environ['MLFLOW_TRACKING_URI'] = 'http://10.147.18.38:31216'
# Now you can use `engine` to interact with your database
engine.connect()

In [3]:
df = pd.read_sql_table('otomoto_data', engine).drop(columns=['index','Oblicz','Kup ten pojazd na ratyOblicz','Pokaż oferty z numerem VIN','Oferta od','Ma numer rejestracyjny'])

df.columns

Index(['Cena', 'Marka pojazdu', 'Model pojazdu', 'Wersja', 'Generacja',
       'Rok produkcji', 'Przebieg', 'Pojemność skokowa', 'Rodzaj paliwa',
       'Moc', 'Skrzynia biegów', 'Napęd', 'Spalanie Poza Miastem',
       'Spalanie W Mieście', 'Typ nadwozia', 'Emisja CO2', 'Liczba drzwi',
       'Liczba miejsc', 'Kolor', 'Rodzaj koloru', 'Kraj pochodzenia',
       'Zarejestrowany w Polsce', ' Pierwszy właściciel (od nowości)',
       'Bezwypadkowy', 'Serwisowany w ASO', 'Stan',
       'Data pierwszej rejestracji w historii pojazdu',
       'Numer rejestracyjny pojazdu', 'Wyświetl VIN', 'Tuning', 'Leasing',
       'Opłata początkowa', 'Miesięczna rata', 'Liczba pozostałych rat',
       'Wartość wykupu', 'Spalanie W Cyklu Mieszanym',
       'Możliwość finansowania', 'Faktura VAT', 'lub do (przebieg km)',
       'Uszkodzony', 'VAT marża', 'Kierownica po prawej (Anglik)', 'Autonomia',
       'Średnie zużycie', 'Rodzaj własności baterii'],
      dtype='object')

In [ ]:
['Cena', 'Marka pojazdu', 'Model pojazdu', 'Wersja', 'Generacja',
'Rok produkcji', 'Przebieg', 'Pojemność skokowa', 'Rodzaj paliwa',
'Moc', 'Skrzynia biegów', 'Napęd', 'Spalanie W Mieście', 'Typ nadwozia','Liczba drzwi',
'Liczba miejsc', 'Kolor', 'Kraj pochodzenia','Stan']

In [ ]:
['Price', 'Vehicle brand', 'Vehicle model', 'Vehicle Version', 'Vehicle Generation',
'Year of production', 'Mileage', 'Engine Capacity', 'Fuel Type', 'Horse Power',
'Transmission Type', 'Drive Type', 'Gas Usage per 100km', 'Car Body Type',
'Number of doors', 'Number of seats', 'Color', 'Country of origin',
'New/Used']


In [7]:

from sqlalchemy import create_engine
import os
import pandas as pd
from googletrans import Translator
from tqdm import tqdm
import numpy as np
import logging
from typing import Union
from sqlalchemy import create_engine
from typing import Any


logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


class Preprocessing:
    def __init__(self):
        self.engine = create_engine(os.getenv('DB_CONNECTION_STRING'))
        self.df = pd.read_sql_table('otomoto_data',con=self.engine)\
            .drop(columns=['index','Oblicz','Kup ten pojazd na ratyOblicz','Pokaż oferty z numerem VIN','Oferta od','Ma numer rejestracyjny'])


    @staticmethod
    def _remove_na_columns(df: pd.DataFrame) -> pd.DataFrame:
        n_total = len(df)
        for col in df.columns:
            n_nulls = len(df.loc[df[col].isna()])
            if n_nulls/n_total > 0.35:
                df = df.drop(columns=[col])
        return df


    @staticmethod
    def _drop_rows_with_nulls(df: pd.DataFrame) -> pd.DataFrame:
        df = df.dropna(subset=['Cena', 'Przebieg', 'Pojemność skokowa', 'Moc'])
        return df


    @staticmethod
    def _convert_to_type(x: Any, type: Union[int, float, str]) -> Union[int, float, str]:
        try:
            return type(x)
        except Exception:
            return np.nan


    def _convert_datatypes(self, df: pd.DataFrame) -> pd.DataFrame:
        df['Rok produkcji'] = df['Rok produkcji'].apply(lambda x: self._convert_to_type(x, int))
        df['Przebieg'] = df['Przebieg'].astype(str).str.replace(' km', '').str.replace(' ', '').astype(float)
        df['Pojemność skokowa'] = df['Pojemność skokowa'].str.replace(' cm3', '').str.replace(' ', '').astype(float)
        df['Moc'] = df['Moc'].str.replace(' KM', '').str.replace(' ', '').astype(float)
        df['Spalanie W Mieście'] = df['Spalanie W Mieście'].str.replace(' l/100km', '').str.replace(',', '.').str.replace(' ', '').astype(float)
        df['Liczba drzwi'] = df['Liczba drzwi'].apply(lambda x: self._convert_to_type(x, int))
        df['Liczba miejsc'] = df['Liczba miejsc'].apply(lambda x: self._convert_to_type(x, int))
        return df

    @staticmethod
    def _translate_to_eng(df: pd.DataFrame) -> pd.DataFrame:
        translator = Translator()
        translator = Translator(service_urls=[
            'translate.google.com',
            'translate.google.pl',
            ])

        columns_to_translate = ['Rodzaj paliwa', 'Skrzynia biegów', 'Napęd', 'Typ nadwozia', 'Kolor', 'Kraj pochodzenia', 'Stan']
        logger.info("Translating column contents...")
        for col in tqdm(columns_to_translate):
            if col in df.columns:
                unique_values = df[col].dropna().unique()
                translations = {}
                for value in unique_values:
                    try:
                        translations[value] = translator.translate(value).text
                    except AttributeError:
                        translations[value] = value
                df[col] = df[col].map(translations)

        df.columns = ['Price', 'Vehicle brand', 'Vehicle model', 'Vehicle Version', 'Vehicle Generation',
            'Year of production', 'Mileage', 'Engine Capacity', 'Fuel Type', 'Horse Power',
            'Transmission Type', 'Drive Type', 'Gas Usage per 100km', 'Car Body Type',
            'Number of doors', 'Number of seats', 'Color', 'Country of origin',
            'New/Used']

        return df


    def preprocess_data(self) -> pd.DataFrame:
        self.df = self._remove_na_columns(self.df)
        self.df = self._drop_rows_with_nulls(self.df)
        self.df = self._convert_datatypes(self.df)
        self.df = self._translate_to_eng(self.df)
        
        self.save_to_sql('otomoto_data_cleaned', os.getenv('DB_CONNECTION_STRING'))
        return self.df


    def get_data(self) -> pd.DataFrame:

        self.df.columns = [
            'price', 'vehicle_brand', 'vehicle_model', 'vehicle_version',
            'vehicle_generation', 'year_of_production', 'mileage',
            'engine_capacity', 'fuel_type', 'horse_power', 'transmission_type',
            'drive_type', 'gas_usage_per_100km', 'car_body_type', 'number_of_doors',
            'number_of_seats', 'color', 'country_of_origin', 'new_used'
        ]

        return self.df


    def save_to_sql(self, table_name: str, sql_engine: str):
        engine = create_engine(sql_engine)
        self.df.to_sql(table_name, engine, if_exists='replace', index=False)

In [8]:
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import mlflow
import mlflow.sklearn
import datetime
from warnings import filterwarnings
import os
filterwarnings('ignore')


class ModelTrainer:
    def __init__(self, df, target_column):
        self.mlflow_uri = os.getenv('MLFLOW_TRACKING_URI')
        self.df = df
        self.target_column = target_column
        self.models = [('RandomForestRegressor', RandomForestRegressor()), 
                       ('XGBRegressor', XGBRegressor()), 
                       ('LGBMRegressor', LGBMRegressor())]
        self.parameters = {'n_estimators': [50, 100, 200], 
                           'max_depth': [6, 10, 15], 
                           'min_samples_split': [2, 5, 10]}
        self.encoder = OneHotEncoder(handle_unknown='ignore')

    def preprocess(self):
        mlflow.set_tracking_uri(self.mlflow_uri)
        mlflow.set_experiment("otomoto_price_predictor")

        X = self.df.drop(self.target_column, axis=1)
        y = self.df[self.target_column]

        with mlflow.start_run() as encoder_run:
            run_name = datetime.datetime.now().strftime("%Y-%m-%d %H:%M") 
            mlflow.set_tag("mlflow.runName", run_name + '_encoder')
            self.encoder.fit(X)
            mlflow.sklearn.log_model(self.encoder, "otomoto_data_encoder")
            mlflow.register_model("runs:/{}/otomoto_data_encoder".format(mlflow.active_run().info.run_id), "otomoto_data_encoder")
        mlflow.end_run()

        X_encoded = self.encoder.transform(X).toarray()
        y = y.values

        self.train, self.test, self.y_train, self.y_test = train_test_split(X_encoded, y, test_size=0.3, random_state=42)
        self.valid, self.test, self.y_valid, self.y_test = train_test_split(self.test, self.y_test, test_size=0.33, random_state=42)

    def train_models(self):
        mlflow.set_tracking_uri(self.mlflow_uri)
        mlflow.set_experiment("otomoto_price_predictor")

        for model_name, model in self.models:
            with mlflow.start_run() as run:
                run_name = datetime.datetime.now().strftime("%Y-%m-%d %H:%M")    
                mlflow.set_tag("mlflow.runName", run_name)

                cv = KFold(n_splits=5, random_state=42, shuffle=True)
                grid_cv = GridSearchCV(model, self.parameters, cv=cv)
                model_name = model_name + '_otomoto_price_predictor'

                grid_cv.fit(self.train, self.y_train)

                for name, dataset, y_true in [('Train', self.train, self.y_train), ('Validation', self.valid, self.y_valid), ('Test', self.test, self.y_test)]:
                    pred = grid_cv.predict(dataset)
                    r2 = r2_score(y_true, pred)
                    mae = mean_absolute_error(y_true, pred)
                    rmse = np.sqrt(mean_squared_error(y_true, pred))
                    mlflow.log_metric(f'{model_name} {name} R2', r2)
                    mlflow.log_metric(f'{model_name} {name} MAE', mae)
                    mlflow.log_metric(f'{model_name} {name} RMSE', rmse)

                params = {f"{model_name}_{k}": v for k, v in grid_cv.best_params_.items()}
                mlflow.log_params(params)

                model_info = mlflow.sklearn.log_model(grid_cv.best_estimator_, "model")
                model_uri = "runs:/" + run.info.run_id + "/" + model_info.artifact_path
                mlflow.register_model(model_uri, f"{model_name}")

In [9]:
data_processor = Preprocessing()
data_processor.preprocess_data()

input_data = data_processor.get_data()

# model_trainer = ModelTrainer(input_data, target_column='price')
# model_trainer.preprocess()
# model_trainer.train_models()

KeyError: 'Spalanie W Mieście'

In [ ]:
import pandas as pd
import mlflow.sklearn


class Predictor:
    def __init__(self, model_name, transformer_name):
        self.model_name = model_name
        self.transformer_name = transformer_name
        self.mlflow_client = mlflow.tracking.MlflowClient()

    def load_model(self):
        model_version = self.mlflow_client.get_latest_versions(self.model_name)[0]
        self.model = mlflow.sklearn.load_model(model_uri=f"models:/{self.model_name}/{model_version.version}")

    def load_transformer(self):
        transformer_version = self.mlflow_client.get_latest_versions(self.transformer_name)[0]
        self.transformer = mlflow.sklearn.load_model(model_uri=f"models:/{self.transformer_name}/{transformer_version.version}")

    def load_models(self):
        mlflow.set_tracking_uri(os.getenv('MLFLOW_TRACKING_URI'))
        mlflow.set_experiment("otomoto_price_predictor")
        self.load_model()
        self.load_transformer()

    def predict(self, data_dict):
        data_df = pd.DataFrame([data_dict])
        data_transformed = self.transformer.transform(data_df)
        prediction = self.model.predict(data_transformed)

        return prediction[0]

In [36]:
import requests
import json
sample_vehicle = {
    "model_name": "RandomForestRegressor_otomoto_price_predictor",
    "vehicle_brand": "Toyota",
    "vehicle_model": "Corolla",
    "vehicle_version": "1.6 VVT-i",
    "vehicle_generation": "E120",
    "year_of_production": 2005,
    "mileage": 150000.0,
    "engine_capacity": 1.6,
    "fuel_type": "Petrol",
    "horse_power": 110.0,
    "transmission_type": "Manual",
    "drive_type": "FWD",
    "gas_usage_per_100km": 6.7,
    "car_body_type": "Sedan",
    "number_of_doors": 4,
    "number_of_seats": 5.0,
    "color": "Silver",
    "country_of_origin": "Japan",
    "new_used": "Used"
}

requests.post('http://localhost:8000/api/v1/otomoto/predict', json=sample_vehicle).json()

{'message': 34454.33975910263}

In [ ]:
pred = Predictor(model_name='RandomForestRegressor_otomoto_price_predictor', transformer_name='otomoto_data_encoder')
pred.load_model()
pred.load_transformer()
pred.predict(sample_vehicle)

In [ ]:
encoder_model_name = 'otomoto_data_encoder'
encoder_model_version = 'latest'

encoder = mlflow.sklearn.load_model(model_uri=f"models:/{encoder_model_name}/{encoder_model_version}")

In [ ]:
data_df = pd.DataFrame([sample_vehicle])

In [ ]:
data_df

In [ ]:
encoder.transform(data_df)

In [ ]:
'DATASET LOADING AND PREPROCESSING'
'='* 50

df = pd.read_sql_table('otomoto_data',con=engine).drop(columns=['index','Oblicz','Kup ten pojazd na ratyOblicz','Pokaż oferty z numerem VIN','Oferta od','Ma numer rejestracyjny'])
df.head()

import numpy as np
import pandas as pd

n_total = len(df)
for col in df.columns:
    n_nulls = len(df.loc[df[col].isna()])
    if n_nulls/n_total > 0.35:
        df = df.drop(columns=[col])

def drop_rows_with_nulls(df):
    # If any of the following columns is null, drop the row: [Cena, Przebieg, Pojemność skokowa, Moc, Spalanie W Mieście]
    df = df.dropna(subset=['Cena', 'Przebieg', 'Pojemność skokowa', 'Moc', 'Spalanie W Mieście'])
    return df


def convert_to_type(x, type: int|float|str):
    try:
        return type(x)
    except:
        return np.nan

df = drop_rows_with_nulls(df)

from googletrans import Translator
from tqdm import tqdm
# Create a translator object
translator = Translator()
translator = Translator(service_urls=[
      'translate.google.com',
      'translate.google.pl',
    ])

columns_to_translate = ['Rodzaj paliwa', 'Skrzynia biegów', 'Napęd', 'Typ nadwozia', 'Kolor', 'Kraj pochodzenia', 'Stan']
print("Translating column contents...")
for col in tqdm(columns_to_translate):
    if col in df.columns:
        unique_values = df[col].dropna().unique()
        translations = {}
        for value in unique_values:
            try:
                translations[value] = translator.translate(value).text
            except AttributeError:
                print(f"Translation failed for value: {value}")
                translations[value] = value  # Use original value if translation fails
        df[col] = df[col].map(translations)

print("Translating column names...")
df.columns = ['Price', 'Vehicle brand', 'Vehicle model', 'Vehicle Version', 'Vehicle Generation',
        'Year of production', 'Mileage', 'Engine Capacity', 'Fuel Type', 'Horse Power',
        'Transmission Type', 'Drive Type', 'Gas Usage per 100km', 'Car Body Type',
        'Number of doors', 'Number of seats', 'Color', 'Country of origin',
        'New/Used']



In [ ]:
'MODEL TRAINING'
'='* 50
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import numpy as np
import mlflow
import mlflow.sklearn
import datetime
from warnings import filterwarnings

filterwarnings('ignore')

from sklearn.preprocessing import OneHotEncoder

# Fit a OneHotEncoder on the original DataFrame
encoder = OneHotEncoder(handle_unknown='ignore')
encoder.fit(df)

def make_prediction(new_data: pd.DataFrame, model):
    # Transform the new data using the fitted encoder
    new_data_encoded = encoder.transform(new_data)

    # Make a prediction using the model
    prediction = model.predict(new_data_encoded)

    return prediction


import mlflow
import joblib
from sklearn.preprocessing import OneHotEncoder

# Fit a OneHotEncoder on the original DataFrame
encoder = OneHotEncoder(handle_unknown='ignore')
encoder.fit(df)

with mlflow.start_run():
    run_name = datetime.datetime.now().strftime("%Y-%m-%d %H:%M") 
    mlflow.set_tag("mlflow.runName", run_name + '_encoder')
    mlflow.sklearn.log_model(encoder, "otomoto_data_encoder")
    mlflow.register_model("runs:/{}/otomoto_data_encoder".format(mlflow.active_run().info.run_id), "otomoto_data_encoder")

mlflow.end_run()
    

df_encoded = encoder.fit_transform(df).toarray()


# 2. Train, validation and test split
train, test = train_test_split(df_encoded, test_size=0.3, random_state=42)
valid, test = train_test_split(test, test_size=0.33, random_state=42)

# 4. Create mapping of sample parameters and do a grid search CV with KFold validation
parameters = {'n_estimators': [50, 100, 200], 'max_depth': [6, 10, 15], 'min_samples_split': [2, 5, 10]}

# Set MLflow server
mlflow.set_tracking_uri("http://10.147.18.38:31216/")

# Set experiment
mlflow.set_experiment("otomoto_price_predictor")

for model_name, model in [('RandomForestRegressor', RandomForestRegressor()), ('XGBRegressor', XGBRegressor()), ('LGBMRegressor', LGBMRegressor())]:
    with mlflow.start_run() as run:
        # Set run name
        run_name = datetime.datetime.now().strftime("%Y-%m-%d %H:%M")    
        mlflow.set_tag("mlflow.runName", run_name)

        cv = KFold(n_splits=5, random_state=42, shuffle=True)
        grid_cv = GridSearchCV(model, parameters, cv=cv)
        model_name = model_name + '_otomoto_price_predictor'

        # 5. Fit model
        X_train, y_train = train[:, :-1], train[:, -1]
        grid_cv.fit(X_train, y_train)

        # 6. Calculate R2, MAE, RMSE on each of those sets
        for name, dataset in [('Train', train), ('Validation', valid), ('Test', test)]:
            X, y = dataset[:, :-1], dataset[:, -1]
            pred = grid_cv.predict(X)
            r2 = r2_score(y, pred)
            mae = mean_absolute_error(y, pred)
            rmse = np.sqrt(mean_squared_error(y, pred))
            mlflow.log_metric(f'{model_name} {name} R2', r2)
            mlflow.log_metric(f'{model_name} {name} MAE', mae)
            mlflow.log_metric(f'{model_name} {name} RMSE', rmse)

        # Log parameters
        params = {f"{model_name}_{k}": v for k, v in grid_cv.best_params_.items()}
        mlflow.log_params(params)

        # Log model
        model_info = mlflow.sklearn.log_model(grid_cv.best_estimator_, "model")

        # Register model
        model_uri = "runs:/" + run.info.run_id + "/" + model_info.artifact_path
        mlflow.register_model(model_uri, f"{model_name}")



In [ ]:
import mlflow.sklearn

with mlflow.start_run():
    run_name = datetime.datetime.now().strftime("%Y-%m-%d %H:%M") 
    mlflow.set_tag("mlflow.runName", run_name + '_encoder')
    mlflow.sklearn.log_model(encoder, "Otomoto_data_encoder")
    mlflow.register_model("runs:/{}/Otomoto_data_encoder".format(mlflow.active_run().info.run_id), "Otomoto_data_encoder")

In [ ]:
encoder.fit_transform(df).toarray()

In [ ]:
df.columns

In [ ]:
'PREDICTIONS'
'='* 50

import mlflow
from mlflow.tracking import MlflowClient

# Initialize MLflow client
client = MlflowClient()

# Define the names of the registered models
model_names = ['RandomForestRegressor', 'XGBRegressor', 'LGBMRegressor']
task = '_otomoto_price_predictor'
model_names = [model_name + task for model_name in model_names]
# Initialize a dictionary to store the models
models = {}

# Load the latest version of each model
for model_name in model_names:

    model_version = client.get_latest_versions(model_name)[0]
    models[model_name] = mlflow.sklearn.load_model(model_uri=f"models:/{model_name}/{model_version.version}")

# Make predictions on the first row of the training set
first_row = train[0, :-1].reshape(1, -1)

for model_name, model in models.items():
    prediction = model.predict(first_row)
    print(f"The prediction from {model_name} is {prediction[0]}")

In [ ]:
df.info()

In [ ]:
import random
import string

keys = ['Price', 'Vehicle brand', 'Vehicle model', 'Vehicle Version', 'Vehicle Generation',
        'Year of production', 'Mileage', 'Engine Capacity', 'Fuel Type', 'Horse Power',
        'Transmission Type', 'Drive Type', 'Gas Usage per 100km', 'Car Body Type',
        'Number of doors', 'Number of seats', 'Color', 'Country of origin',
        'New/Used']

# Generate random values
values = [random.randint(1, 100) if key == 'Price' or key == 'Year of production' or key == 'Mileage' or key == 'Horse Power' or key == 'Number of doors' or key == 'Number of seats'
          else ''.join(random.choices(string.ascii_uppercase + string.digits, k=5)) for key in keys]

# Create a dictionary
data_dict = dict(zip(keys, values))

print(data_dict)

In [ ]:
import mlflow
from mlflow.tracking import MlflowClient

# Initialize MLflow client
client = MlflowClient()

# Define the names of the registered models
model_names = ['RandomForestRegressor', 'XGBRegressor', 'LGBMRegressor']
task = '_otomoto_price_predictor'
model_names = [model_name + task for model_name in model_names]
# Initialize a dictionary to store the models
models = {}

encoder_model_name = 'otomoto_data_encoder'
encoder_model_version = 'latest'

encoder = mlflow.sklearn.load_model(model_uri=f"models:/{encoder_model_name}/{encoder_model_version}")

input_data = encoder.transform(pd.DataFrame(data_dict, index=[0])).toarray()

for model_name in model_names:

    model_version = client.get_latest_versions(model_name)[0]
    models[model_name] = mlflow.sklearn.load_model(model_uri=f"models:/{model_name}/{model_version.version}")

for model_name, model in models.items():
    prediction = model.predict(input_data)
    print(f"The prediction from {model_name} is {prediction[0]}")
